## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="https://github.com/oac0de/dsc-resilient-distributed-datasets-rdd-lab-onl01-dtsc-ft-052620/blob/master/images/rdd_diagram.png?raw=1" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [3]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [5]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 59kB/s 
     |████████████████████████████████| 204kB 45.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257928 sha256=f17e7cc667dd9337646ab1453b6d92db53885851cc4738e146847fbfc4d03d5b
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [6]:
import os
# /usr/lib/jvm/java-8-openjdk-amd64
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# /content/spark-2.4.5-bin-hadoop2.7
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [7]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [11]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [12]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [14]:
# count
rdd.count()

1000

In [15]:
# first
rdd.first()

1

In [17]:
# take
rdd.take(8)

[1, 2, 3, 4, 5, 6, 7, 8]

In [18]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [19]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [20]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([4.51538564e-01, 3.16965127e-01, 2.26947016e+00, 1.65192274e+00,
       3.31478850e+00, 5.88051892e+00, 5.65771453e+00, 2.85212763e+00,
       6.27863330e+00, 1.82591947e+00, 6.36515599e+00, 6.26021764e+00,
       1.06447588e+00, 1.23117712e+01, 1.11505907e+01, 1.14298452e+01,
       5.94367884e+00, 1.44819820e+01, 8.50557641e+00, 1.41567059e+01,
       2.00157659e+01, 1.79981605e+01, 1.26955345e+01, 1.98881460e+01,
       2.26033123e+01, 8.77874331e+00, 2.49029515e+01, 1.99390044e+00,
       2.43543755e+01, 2.52866139e+01, 1.18696341e+01, 1.75898893e+01,
       2.14660500e+01, 1.96156743e+01, 2.48342875e+01, 3.28015287e+01,
       1.60981052e+01, 1.48604976e+01, 2.06723159e+01, 3.30562559e+01,
       1.59725650e+01, 1.37446729e+01, 3.64183290e+01, 9.94257448e+00,
       3.28557121e+01, 3.03778158e+01, 1.43829659e+01, 8.16756740e+00,
       2.29160646e+01, 4.65928462e+01, 3.47475747e+01, 3.28950062e+01,
       1.72309212e+01, 2.41320080e+01, 3.60911729e+01, 1.34962907e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [21]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(5)


[0.4515385637098397,
 0.31696512721135206,
 2.2694701616335458,
 1.6519227399773997,
 3.31478849803057]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [22]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [23]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.41541547861305256,
 0.2916079170344439,
 2.087912548702862,
 1.5197689207792078,
 3.0496054181881247,
 5.410077402747198,
 5.205097370796906,
 2.623957417414815,
 5.776342632164421,
 1.6798459168694193]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [24]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [25]:
discounted.take(10)

[0.3738739307517473,
 0.2624471253309995,
 1.8791212938325759,
 1.367792028701287,
 2.7446448763693123,
 4.869069662472478,
 4.684587633717216,
 2.3615616756733337,
 5.198708368947979,
 1.5118613251824775]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [26]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[774.8153696284006,
 771.0778446050249,
 763.285249595554,
 748.012101863612,
 746.41522995363,
 731.8531382631043,
 726.9145482011871,
 716.3683517716871,
 713.6900037566317,
 711.0349332408825,
 702.312256926592,
 700.4289413254652,
 697.9165571911764,
 696.7312592083036,
 695.0254294993224]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [27]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [28]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.4515385637098397, 0.3738739307517473), (0.31696512721135206, 0.2624471253309995), (2.2694701616335458, 1.8791212938325759), (1.6519227399773997, 1.367792028701287), (3.31478849803057, 2.7446448763693123), (5.880518916029563, 4.869069662472478), (5.657714533474898, 4.684587633717216), (2.8521276276247987, 2.3615616756733337), (6.278633295830892, 5.198708368947979), (1.8259194748580643, 1.5118613251824775)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [29]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.4515385637098397, 0.3738739307517473, 0.31696512721135206, 0.2624471253309995, 2.2694701616335458, 1.8791212938325759, 1.6519227399773997, 1.367792028701287, 3.31478849803057, 2.7446448763693123]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [30]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

270

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [31]:
selected_items.reduce(lambda x,y: x + y)

124626.91230715552

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [32]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(11, 316.38654712687736),
 (22, 326.86694472399967),
 (11, 328.11995915400524),
 (22, 304.8957590415386),
 (16, 315.3470869452148),
 (8, 351.73501645962915),
 (20, 329.5655556937657)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [35]:
# calculate how much each user spent
sales_data.reduceByKey(lambda x, y: x + y)
sales_data.take(6)

[(11, 316.38654712687736),
 (22, 326.86694472399967),
 (11, 328.11995915400524),
 (22, 304.8957590415386),
 (16, 315.3470869452148),
 (8, 351.73501645962915)]

In [38]:
# sort the users from highest to lowest spenders
sales_data.sortBy(lambda x: x[1],ascending = False).collect()

[(5, 774.8153696284006),
 (28, 771.0778446050249),
 (48, 763.285249595554),
 (10, 748.012101863612),
 (35, 746.41522995363),
 (33, 731.8531382631043),
 (8, 726.9145482011871),
 (45, 716.3683517716871),
 (30, 713.6900037566317),
 (19, 711.0349332408825),
 (50, 702.312256926592),
 (29, 700.4289413254652),
 (49, 697.9165571911764),
 (28, 696.7312592083036),
 (10, 695.0254294993224),
 (50, 691.5215048230779),
 (34, 689.8474814760833),
 (26, 688.3154437303574),
 (3, 687.3127657592688),
 (3, 677.0403459471908),
 (8, 673.3725045622159),
 (7, 669.6211097275265),
 (31, 661.0187141372355),
 (34, 659.4678002698514),
 (46, 656.6961876472321),
 (43, 656.5757616269763),
 (23, 653.3206026872102),
 (1, 652.9925991740806),
 (7, 645.8471924256697),
 (36, 642.2451099854537),
 (34, 636.5438747875211),
 (44, 622.0444292385442),
 (10, 621.0907402087144),
 (29, 616.7039170469529),
 (11, 607.1033510238794),
 (11, 606.958706926105),
 (37, 603.804617001631),
 (13, 601.9367061844413),
 (31, 599.7100477432847),
 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [39]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(35, 13),
 (39, 11),
 (34, 11),
 (37, 10),
 (16, 9),
 (24, 9),
 (11, 8),
 (31, 8),
 (21, 8),
 (2, 8),
 (42, 8),
 (38, 7),
 (18, 7),
 (26, 7),
 (10, 7),
 (22, 6),
 (27, 6),
 (25, 6),
 (12, 6),
 (3, 6),
 (7, 6),
 (23, 6),
 (17, 6),
 (8, 5),
 (20, 5),
 (29, 5),
 (49, 5),
 (41, 5),
 (32, 5),
 (4, 5),
 (36, 4),
 (13, 4),
 (9, 4),
 (33, 4),
 (6, 4),
 (50, 4),
 (44, 4),
 (19, 4),
 (48, 3),
 (1, 3),
 (28, 3),
 (47, 3),
 (14, 3),
 (43, 2),
 (5, 2),
 (45, 2),
 (46, 2),
 (30, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
